전결합 신경망으로 MNIST 손글씨를 분류합니다.

In [1]:
# 실습에 필요한 라이브러리를 불러옵니다.
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import joblib

In [2]:
# github에서 MNIST DATASET 다운로드
!git clone https://github.com/baek2sm/ml.git
!tar -zxvf ./ml/datasets/MNIST.tar.gz

# 현재 경로에 MNIST 학습 세트와 테스트 세트를 불러옵니다.
path = './'
train_dataset = datasets.MNIST(path, train=True, download=True)
test_dataset = datasets.MNIST(path, train=False, download=True)

Cloning into 'ml'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 41 (delta 5), reused 33 (delta 2), pack-reused 0
Unpacking objects: 100% (41/41), done.
MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/train-images-idx3-ubyte.gz
MNIST/processed/
MNIST/processed/training.pt
MNIST/processed/test.pt


In [3]:
# 학습 세트와 테스트 세트의 입력 데이터와 타깃을 준비합니다.
X_train, y_train = train_dataset.data / 255, train_dataset.targets
X_test, y_test = test_dataset.data / 255, test_dataset.targets

In [4]:
# 학습 세트와 테스트 세트의 데이터 형태를 확인합니다.
print('학습 세트 입력 데이터:', X_train.shape)
print('학습 세트 타깃:', y_train.shape)
print('테스트 세트 입력 데이터:', X_test.shape)
print('테스트 세트 타깃:', y_test.shape)


학습 세트 입력 데이터: torch.Size([60000, 28, 28])
학습 세트 타깃: torch.Size([60000])
테스트 세트 입력 데이터: torch.Size([10000, 28, 28])
테스트 세트 타깃: torch.Size([10000])


In [5]:
# 3차원 형태인 입력 데이터를 2차원 형태로 변환합니다.
X_train, X_test = X_train.view(-1, 784), X_test.view(-1, 784)

# 학습 세트와 테스트 세트의 데이터 형태를 확인합니다.
print('학습 세트 입력 데이터:', X_train.shape)
print('테스트 세트 입력 데이터:', X_test.shape)

학습 세트 입력 데이터: torch.Size([60000, 784])
테스트 세트 입력 데이터: torch.Size([10000, 784])


In [6]:
# 입력 데이터와 타깃을 묶어 텐서 데이터 세트를 생성합니다.
train_dset = TensorDataset(X_train, y_train)
test_dset = TensorDataset(X_test, y_test)

# 한 번에 128개의 데이터 샘플을 배치로 사용하는 데이터 로더를 생성합니다.
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=False)

In [7]:
# n개의 값을 입력 받는 DNN 모델 클래스를 정의합니다.
class DNN(nn.Module):
    # 생성자에서 모델의 구조를 정의합니다. 
    def __init__(self, num_features):
        # 상속받아 생성한 객체이므로 부모(nn.Module)의 생성자를 호출합니다.
        super().__init__()
        # 첫 번째 은닉층을 정의합니다.
        self.hidden_layer1 = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU()
        )
        # 두 번째 은닉층을 정의합니다.
        self.hidden_layer2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU()
        )
        # 출력층을 정의합니다.
        self.output_layer = nn.Sequential(
            nn.Linear(128, 10)
        )        

    # 모델의 순전파를 정의합니다.
    def forward(self, X):
        # 생성자에서 만든 은닉층과 출력층 노드로 타깃을 예측하고 반환합니다.
        out = self.hidden_layer1(X)
        out = self.hidden_layer2(out)
        out = self.output_layer(out)
        return out

In [12]:
# 784개의 값을 입력 받는 로지스틱 회귀 모델 객체를 생성합니다.
model = DNN(784).to('cuda')

# 크로스 엔트로피(Cross Entropy) 손실 함수 객체를 생성합니다.
criterion = nn.CrossEntropyLoss().to('cuda')

# 확률적 경사 하강법 옵티마이저 객체를 생성합니다.
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, loader):
  # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
  epoch_loss = 0
  epoch_acc = 0

  # 배치 학습을 실행합니다.
  for X_batch, y_batch in loader:
    # 입력 데이터와 타깃의 배치를 그래픽카드로 연산하도록 설정합니다.
    X_batch, y_batch = X_batch.to('cuda'), y_batch.to('cuda')
    # 기울기를 초기화합니다.
    optimizer.zero_grad()
    # 모델을 사용해 타깃을 예측합니다.
    hypothesis = model(X_batch)
    # 손실 함수로 오차를 계산합니다.
    loss = criterion(hypothesis, y_batch)        
    # 기울기를 계산합니다.
    loss.backward()
    # 경사 하강법으로 가중치를 수정합니다.
    optimizer.step()
    # 정확도를 계산합니다.
    y_predicted = torch.argmax(hypothesis, 1)
    acc = (y_predicted == y_batch).float().mean()
    # 현재 배치의 오차와 정확도를 저장합니다.
    epoch_loss += loss.item()
    epoch_acc += acc.item()

  # 현재 에포크의 오차와 정확도를 반환합니다.
  return epoch_loss / len(loader), epoch_acc / len(loader)

In [10]:
# 평가 함수를 정의합니다.
def evaluate(model, criterion, optimizer, loader):
  # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
  epoch_loss = 0
  epoch_acc = 0

  with torch.no_grad():
    # 배치 학습을 실행합니다.
    for X_batch, y_batch in loader:
      # 입력 데이터와 타깃의 배치를 그래픽카드로 연산하도록 설정합니다.
      X_batch, y_batch = X_batch.to('cuda'), y_batch.to('cuda')
      # 모델을 사용해 타깃을 예측합니다.
      hypothesis = model(X_batch)
      # 손실 함수로 오차를 계산합니다.
      loss = criterion(hypothesis, y_batch)
      # 정확도를 계산합니다.
      y_predicted = torch.argmax(hypothesis, 1)
      acc = (y_predicted == y_batch).float().mean()
      # 현재 배치의 오차와 정확도를 저장합니다.
      epoch_loss += loss.item()
      epoch_acc += acc.item()

    # 현재 에포크의 오차와 정확도를 반환합니다.
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [13]:
# 20회에 걸쳐 모델을 학습합니다.
for epoch in range(1, 21):
  # 모델을 학습시킵니다.
  loss, acc = train(model, criterion, optimizer, train_loader)

  # 모델을 평가합니다.
  test_loss, test_acc = evaluate(model, criterion, optimizer, test_loader)

  # 현재 에포크의 학습 결과를 출력합니다.
  print('epoch: {}, loss: {:.3f}, acc: {:.2f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(
      epoch, loss, acc, test_loss, test_acc
  )) 

epoch: 1, loss: 0.553, acc: 0.86, test_loss: 0.281, test_acc: 0.921
epoch: 2, loss: 0.253, acc: 0.93, test_loss: 0.215, test_acc: 0.939
epoch: 3, loss: 0.198, acc: 0.94, test_loss: 0.174, test_acc: 0.948
epoch: 4, loss: 0.163, acc: 0.95, test_loss: 0.148, test_acc: 0.957
epoch: 5, loss: 0.136, acc: 0.96, test_loss: 0.130, test_acc: 0.962
epoch: 6, loss: 0.117, acc: 0.97, test_loss: 0.118, test_acc: 0.964
epoch: 7, loss: 0.101, acc: 0.97, test_loss: 0.108, test_acc: 0.968
epoch: 8, loss: 0.088, acc: 0.97, test_loss: 0.099, test_acc: 0.970
epoch: 9, loss: 0.078, acc: 0.98, test_loss: 0.092, test_acc: 0.971
epoch: 10, loss: 0.068, acc: 0.98, test_loss: 0.084, test_acc: 0.973
epoch: 11, loss: 0.061, acc: 0.98, test_loss: 0.084, test_acc: 0.975
epoch: 12, loss: 0.053, acc: 0.98, test_loss: 0.078, test_acc: 0.976
epoch: 13, loss: 0.048, acc: 0.99, test_loss: 0.078, test_acc: 0.976
epoch: 14, loss: 0.043, acc: 0.99, test_loss: 0.077, test_acc: 0.977
epoch: 15, loss: 0.038, acc: 0.99, test_los